In [1]:
import pandas as pd
import statistics
df = pd.read_csv('data/wine_cleared.zip')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             129971 non-null  int64  
 1   country                129908 non-null  object 
 2   description            129971 non-null  object 
 3   designation            129971 non-null  object 
 4   points                 129971 non-null  int64  
 5   price                  129971 non-null  float64
 6   province               129908 non-null  object 
 7   region_1               129971 non-null  object 
 8   taster_name            129971 non-null  object 
 9   taster_twitter_handle  129971 non-null  object 
 10  title                  129971 non-null  object 
 11  variety                129970 non-null  object 
 12  winery                 129971 non-null  object 
dtypes: float64(1), int64(2), object(10)
memory usage: 12.9+ MB


В прошлом юните мы рассмотрели вычисление максимального, минимального значения признака как вид одномерного анализа, а также меру центральной тенденции и её виды. В этом юните мы рассмотрим корреляционный анализ как один из часто применяемых средств многомерного анализа данных.

>Корреляция — статистическая связь двух и более переменных. При изменении значения одной из переменных происходит закономерное изменение другой или других величин. 

> Например, при взрослении у человека меняются его показатели: рост становится больше, риск сердечно-сосудистых заболеваний — выше.

Мерой связи величин, мерой корреляции является `коэффициент корреляции`. Он может принимать значение от **-1 до +1**.

`Отрицательная корреляция`, коэффициент корреляции **< 0** говорит об обратной связи между переменными. При обратной связи увеличение одной из величин ведёт к закономерному уменьшению другой (других) величин. 

> Например, чем больше сумма активного долга, тем меньше шанс в одобрении кредита.

`Положительная корреляция`, коэффициент корреляции **> 0** говорит о прямой связи между переменными. При прямой связи увеличение одной из величин ведёт к закономерному увеличению другой (других) величин. 

> Например, с увеличением возраста размер ноги человека увеличивается, и его рост увеличивается тоже.

`Коэффициент корреляции` **= 0** говорит о том, что переменные независимы друг от друга, по крайней мере линейно, но это вовсе не значит, что между ними нет какой-то более сложной взаимосвязи. Это достаточно редкое явление, которое может встретиться на конкретном наборе данных.

Понимание корреляции приводит нас к понятию `мультиколлинеарности` — такой сильной зависимости переменных друг от друга, что она затрудняет анализ и оценку будущей модели машинного обучения. 

Понять, почему это плохо для модели машинного обучения, вы сможете, когда приступите к обучению моделей. Но уже на этапе разведывательного анализа данных вы сможете выявить такие зависимости.

Как понять, существует ли сильная взаимозавимость признаков? Для этого существует примерная шкала степеней силы связи и коэффициента корреляции между признаками.

В таблице отражены основные силы связи между признаками.

<table style="width: 80%; padding: 0.2em; border-collapse: collapse; margin: 0px auto; border: 1px solid #b0b0b0;">
<tbody>
<tr>
<td style="width: 50%; text-align: center; vertical-align: middle; background-color: #10564f; border: 1px solid #b0b0b0; padding: 0.2em;"><span style="color: #ffffff;">Сила связи</span></td>
<td style="width: 50%; text-align: center; vertical-align: middle; background-color: #10564f; border: 1px solid #b0b0b0; padding: 0.2em;"><span style="color: #ffffff;">Значение коэффициента корреляции</span></td>
</tr>
<tr>
<td style="width: 50%; text-align: left; border: 1px solid #b0b0b0; padding: 0.2em;">Отсутствие связи или очень слабая связь</td>
<td style="width: 50%; text-align: center; border: 1px solid #b0b0b0; padding: 0.2em;">0…+/- 0.3</td>
</tr>
<tr>
<td style="width: 50%; text-align: left; border: 1px solid #b0b0b0; padding: 0.2em;">Слабая связь</td>
<td style="width: 50%; text-align: center; border: 1px solid #b0b0b0; padding: 0.2em;">+/- 0.3…+/- 0.5</td>
</tr>
<tr>
<td style="width: 50%; text-align: left; border: 1px solid #b0b0b0; padding: 0.2em;">Средняя связь</td>
<td style="width: 50%; text-align: center; border: 1px solid #b0b0b0; padding: 0.2em;">+/- 0.5…+/- 0.7</td>
</tr>
<tr>
<td style="width: 50%; text-align: left; border: 1px solid #b0b0b0; padding: 0.2em;">Сильная связь</td>
<td style="width: 50%; text-align: center; border: 1px solid #b0b0b0; padding: 0.2em;">+/- 0.7…+/- 0.9</td>
</tr>
<tr>
<td style="width: 50%; text-align: left; border: 1px solid #b0b0b0; padding: 0.2em;">Очень сильная или абсолютная связь</td>
<td style="width: 50%; text-align: center; border: 1px solid #b0b0b0; padding: 0.2em;">+/- 0.9…+/-1</td>
</tr>
</tbody>
</table>

> Принято считать, что при коэффициенте корреляции **+/-0,7** связь между признаками сильная и приводит к мультиколлинеарности.

Например, сложилась ситуация, когда в датасете были обнаружены признаки с очень сильной корреляцией. Мы знаем, что мультиколлинеарность вредит такой модели, поэтому на этапе проектирования признаков можем объединить эту пару признаков в один или удалить из этой пары один — тот, что менее важен. Важность признаков мы оцениваем позже, на этапе отбора признаков.

Так что для этапа корреляционного анализа достаточно делать и запоминать выводы о взаимосвязях признаков, чтобы на дальнейшем этапе разведывательного анализа данных правильно их обработать. Эти этапы мы изучим позже в курсе, в юнитах про разведывательный анализ данных.

Для расчёта коэффициента корреляции применим функцию [df.corr()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) библиотеки pandas.

In [2]:
corr = df.corr(numeric_only=True)

print(corr)

            Unnamed: 0    points     price
Unnamed: 0    1.000000  0.007124  0.004652
points        0.007124  1.000000  0.402137
price         0.004652  0.402137  1.000000


Результатом функции `df.corr()` является матрица корреляции. 

Матрица корреляции — таблица, заголовками и строками которой являются названия признаков в датасете. На пересечении строк и столбцов находится значение коэффициента корреляции этих двух признаков.

Данная матрица имеет значение 1 по главной диагонали, так как единица означает корреляцию признака с самим собой. Матрица является симметричной. 

<center><img src='img/dst-eda-2-3.png'></center>

***Визуализация матрицы корреляций на примере датасета о пассажирах «Титаника». Источник: kaggle***

На рисунке вы можете увидеть визуализированную матрицу к одному из соревнований на *kaggle*. Наибольшая связь наблюдается в паре признаков *age* и *age_bucket* — 0.98, что по шкале корреляции является очень сильной связью и приводит к ***мультиколлинеарности*** признаков. Скорее всего, один из признаков будет исключён из датасета.

> Подобную матрицу мы научимся строить в одном из следующих юнитов, посвящённых визуализации корреляций.

В нашей матрице датасета винных обзоров отражено всего два признака — `points`, `price`.  На пересечении признаков мы видим значение коэффициента корреляции между признаками `points` и `price`.

<center><img src='img/dst-eda-2-4.png'></center>

***Матрица корреляции. построенная при помощи df.corr(). Зелёным выделена главная диагональ.***

Причина в том, что рассчитать коэффициент корреляции мы можем только в том случае, когда признак имеет численное выражение. Как мы выяснили ранее, остальные признаки — в строковом формате, и расчёт коэффициента корреляции невозможен. Для этого необходимо провести кодирование признака, которое мы будем изучать далее.

Поскольку наша задача — предсказание рейтинга (баллов) за вино на основании признаков в датасете, можем ли мы с помощью корреляции понять, влияет ли, например, цена за бутылку в определении рейтинга вина?

## Нет!

> Корреляция и причинно-следственная связь — это понятия, которые чаще всего ошибочно приравниваются друг к другу. Понимание корреляции важно для разведывательного анализа и помогает делать правильные выводы. 

#### Ложная корреляция

Рассмотрим, почему корреляция не подразумевает причинно-следственных связей, на популярном примере.

> Продажи мороженого коррелируют с убийствами!
По мере роста или падения продаж замороженного десерта соответственно растёт или уменьшается количество убийств. Является ли потребление замороженного десерта причиной смерти людей?

Нет. Эти две вещи взаимосвязаны, но это не значит, что одна вызывает другую.

Этот случай можно назвать совпадением, и чаще всего корреляция наблюдается именно в таких ситуациях. А иногда нам просто не хватает данных, чтобы уловить причину связи между переменными.

В некоторых случаях участвуют скрытые факторы. Например, в примере с мороженым улучшение погоды вызывает рост продаж десерта, а большое количество людей на улицах создаёт благоприятные условия для убийц.

Нет причинно-следственной связи между убийствами и продажей мороженого, а наличие третьего фактора — хорошей погоды — объясняет эти связи. Этот случай называется `ложной корреляцией`.

> Существует огромное количество подобных исследований с удивительными связями. Прочитайте, например, про [связь пиратов с глобальным потеплением](https://ru.wikipedia.org/wiki/Пастафарианство#Пираты_и_глобальное_потепление). Больше исследований вы найдёте [здесь](http://www.tylervigen.com/spurious-correlations).

В этом юните мы узнали, что такое `корреляция`, `коэффициент корреляции` и `сила связи переменных`, а также `ложная корреляция`. В следующих юнитах вы узнаете о типах корреляции и их разнице.